In [1]:
### Preparacion del dataset inicial
import pandas as pd
import numpy as np
import os
import sys
import re
import argparse

In [2]:
def cargar_y_combinar_datos():
    # Cargar archivos
    sell_in = pd.read_csv('sell-in.txt', sep='\t')
    stocks = pd.read_csv('tb_stocks.txt', sep='\t')
    productos = pd.read_csv('tb_productos.txt', sep='\t')
    #drop duplicates in productos
    productos = productos.drop_duplicates(subset=['product_id'])

    df = sell_in.merge(stocks, on=['periodo', 'product_id'], how='left')
    df = df.merge(productos, on='product_id', how='left')

    return df

def transformar_periodo(df):
    df['fecha'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
    return df

def completar_series_temporales(df,umbral_meses=8):
    """
    Completa las series temporales de productos y clientes cuando AMBOS estaban activos.
    Un cliente o producto se considera 'MUERTO' si entre la ultima fecha del dataset y su ultima fecha de compra pasaron mas de X meses definido por un umbral.
    Para el resto, se considera activo desde su primera compra hasta el fin del dataset.
    """
    import pandas as pd

    df['fecha'] = df['fecha'].astype('period[M]')

    columnas_a_rellenar = ['cust_request_qty', 'cust_request_tn', 'tn']

    # Última fecha del dataset
    fecha_max_dataset = df['fecha'].max()

    # Definir umbral de inactividad
    umbral_fecha = fecha_max_dataset - umbral_meses

    # Fechas de inicio y fin por cliente y producto
    fechas_clientes = df.groupby('customer_id')['fecha'].agg(fecha_ini_c='min', fecha_fin_c='max').reset_index()
    fechas_productos = df.groupby('product_id')['fecha'].agg(fecha_ini_p='min', fecha_fin_p='max').reset_index()

    # Determinar cuáles clientes y productos están inactivos
    fechas_clientes['inactivo'] = fechas_clientes['fecha_fin_c'] <= umbral_fecha
    fechas_productos['inactivo'] = fechas_productos['fecha_fin_p'] <= umbral_fecha

    # Si está inactivo, se respeta la última fecha real; si no, se extiende hasta el final del dataset
    fechas_clientes['fecha_fin_c'] = fechas_clientes.apply(
        lambda x: x['fecha_fin_c'] if x['inactivo'] else fecha_max_dataset, axis=1
    )
    fechas_productos['fecha_fin_p'] = fechas_productos.apply(
        lambda x: x['fecha_fin_p'] if x['inactivo'] else fecha_max_dataset, axis=1
    )

    # Crear rango completo de fechas en formato Period[M]
    fechas = pd.period_range(df['fecha'].min(), fecha_max_dataset, freq='M')
    fechas_df = pd.DataFrame({'fecha': fechas})

    # Crear combinaciones válidas cliente-fecha
    clientes_fechas = fechas_clientes.merge(fechas_df, how='cross')
    clientes_fechas = clientes_fechas[
        (clientes_fechas['fecha'] >= clientes_fechas['fecha_ini_c']) &
        (clientes_fechas['fecha'] <= clientes_fechas['fecha_fin_c'])
    ][['customer_id', 'fecha']]

    # Crear combinaciones válidas producto-fecha
    productos_fechas = fechas_productos.merge(fechas_df, how='cross')
    productos_fechas = productos_fechas[
        (productos_fechas['fecha'] >= productos_fechas['fecha_ini_p']) &
        (productos_fechas['fecha'] <= productos_fechas['fecha_fin_p'])
    ][['product_id', 'fecha']]

    # Crear combinaciones cliente-producto-fecha donde ambos estaban activos
    combinaciones_validas = productos_fechas.merge(clientes_fechas, on='fecha', how='inner')

    # Merge con el dataset original
    df_completo = combinaciones_validas.merge(df, on=['product_id', 'customer_id', 'fecha'], how='left')

    # Completar valores faltantes con 0 en las columnas de cantidad
    df_completo[columnas_a_rellenar] = df_completo[columnas_a_rellenar].fillna(0)

    # Ordenar por producto, cliente y fecha
    df_completo = df_completo.sort_values(['product_id', 'customer_id', 'fecha'])

    return df_completo.reset_index(drop=True)

def crear_variable_objetivo(df):
    # Ordenar dataset por producto, cliente y fecha:
    df = df.sort_values(['product_id', 'customer_id', 'fecha'])
    df['target'] = df.groupby(['product_id', 'customer_id'])['tn'].shift(-2)
    return df

def crear_variable_objetivo_delta(df):
    # Ordenar dataset por producto, cliente y fecha:
    df = df.sort_values(['product_id', 'customer_id', 'fecha'])
    # tn en t+2
    tn_shift_2 = df.groupby(['product_id', 'customer_id'])['tn'].shift(-2)
    # Delta: tn en t menos tn en t+2
    df['target_delta'] = df['tn'] - tn_shift_2
    return df

### TARGET T+2

In [3]:
df = cargar_y_combinar_datos()
df = transformar_periodo(df)
df = completar_series_temporales(df, umbral_meses=10)
df = crear_variable_objetivo(df)
df.info()

KeyboardInterrupt: 

In [ ]:
df.head()
df.to_parquet("df_inicial.parquet")

### TARGET DELTA TN

In [5]:
df = cargar_y_combinar_datos()
df = transformar_periodo(df)
df = completar_series_temporales(df, umbral_meses=10)
df = crear_variable_objetivo_delta(df)
df.to_parquet("df_inicial_target_delta.parquet")

In [6]:
df.head()

,product_id,fecha,customer_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,cat3,brand,sku_size,target_delta
0,20001,2017-01,10001,201701.0,0.0,11.0,99.43861,99.43861,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,6.97324
1,20001,2017-02,10001,201702.0,0.0,23.0,198.84365,198.84365,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,185.54637
2,20001,2017-03,10001,201703.0,0.0,33.0,92.46537,92.46537,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,-8.54026
3,20001,2017-04,10001,201704.0,0.0,8.0,13.29728,13.29728,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,-114.75064
4,20001,2017-05,10001,201705.0,0.0,15.0,101.20711,101.00563,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,-0.20148
